In [ ]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir data
!cp '/content/drive/My Drive/pred-analit/GC-predictive-analytics-worksheet - Sheet1.csv' data

In [ ]:
df = pd.read_csv('data/GC-predictive-analytics-worksheet - Sheet1.csv')

In [ ]:
rec_idxs = [str(i) for i in range(1, 16)]
er_idxs = ['ER']
er_idxs.extend(['ER.{}'.format(i) for i in range(1, 15)])

In [ ]:
rrecs = []
eers = []
iids = []
for i in range(len(df)):
  recs = []
  ers = []
  for rec, er in zip(rec_idxs, er_idxs):
      recomendation = df[rec][i]
      try:
        engagement = float(df[er][i])
      except ValueError:
        engagement = 0.0
      if pd.isna(recomendation):
        continue
      elif recomendation == 'Phy':
        recomendation = 0
      else:
        recomendation = 1
      recs.append(recomendation)
      ers.append(engagement)
  rrecs.append(recs)
  eers.append(ers)
  gender = df.Gender[i]
  hours = df['Work hours'][i]
  hmap = {'1-15 hrs/week': 0.2, '16-35 hrs/week': 0.4, '36-50 hrs/week': 0.6, '50+ hrs/week': 1.0}
  if hours in hmap:
    hours = hmap[hours]
  else:
    hours = 0.0
  if gender == 'Female':
    gender = 0
  else:
    gender = 1
  motivation = df.Motivation[i]	
  if pd.isna(motivation):
    motivation = 0.0
  attitude = df.Attitude[i]
  if pd.isna(attitude):
    attitude = 0
  intention = df.Intention[i]
  if pd.isna(intention):
    intention = 0
  ownership = df.Ownership[i]
  if pd.isna(ownership):
    ownership = 0
  iids.append(dict(age = df.Age[i] / 82,
                   gender = gender,  
                   motivation = motivation,	
                   attitude = attitude, 
                   intention = intention,
                   ownership = ownership,
                   hours = hours))

In [ ]:
X = []
y = []
for i in range(len(rrecs)):
  for j in range(len(rrecs[i]) - 2):
    r1 = rrecs[i][j]
    r2 = rrecs[i][j+1]
    # X.append([eers[i][j], r2 - r1, r1, r2, iids[i]['age']])
    X.append([eers[i][j], r1, r2, r2-r1, iids[i]['age'], iids[i]['motivation']]) #iids[i]['attitude']])
    y.append(eers[i][j+1])

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)

In [ ]:
model = LinearRegression()
model.fit(X_, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
pX = []
my_recs = []
for i in [0, 2]:
    r1 = rrecs[i][-1]
    pX.append([eers[i][-1], r1, 1 - r1, 1, iids[i]['age'], iids[i]['motivation']])
    my_recs.append('diet')
    pX.append([eers[i][-1], r1, 0 - r1, 0, iids[i]['age'], iids[i]['motivation']])
    my_recs.append('Phy')

In [ ]:
pX_ = poly.fit_transform(pX)

In [ ]:
model.predict(pX_)

array([0.23776317, 0.22633416, 0.87826127, 0.32420668])

In [ ]:
my_recs

['diet', 'Phy', 'diet', 'Phy']